In [1]:
import pandas as pd
import numpy as np
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1%}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# add trip type to trip tables
def prep_trip_df(trip):
    """Mode shares will represent shares across Drive Alone, Shared Ride, 
       public transit, and walk and bike trips only. 
    """
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    # Exclude TNC, School Bus, and Other modes
    df_trip = df_trip[~df_trip['Mode'].isin(['TNC', 'School Bus', 'Other'])]

    # Aggregate modes
    df_trip['Mode'] = df_trip['Mode'].replace({
        'SOV': 'Drove Alone',
        'HOV2': 'Shared Ride',
        'HOV3+': 'Shared Ride',
        'Transit': 'Transit',
        'Walk': 'Walk',
        'Bike': 'Bike',
    })

    return df_trip

In [4]:
person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')

# counties
trip_county = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_county.csv'))
# remove "Outside Region"
trip_county = trip_county[trip_county['hh_county'] != 'Outside Region']
# regional geographies
trip_rg = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_rg.csv'))
# centers
trip_rgc = pd.read_csv(output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

In [5]:
# equity geographies
equity_geogs = summary_config['hh_equity_geogs']
trip_equity_geog = pd.DataFrame()
for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])


## regional mode share

In [6]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df.pivot_table(columns='Mode', index='Trip Type', values='mode_share')

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
Trip Type,,,,,
All Trips,1.5%,45.4%,36.1%,2.2%,14.7%
Non-Work,1.6%,41.2%,39.4%,2.2%,15.6%
Work,1.4%,72.0%,15.3%,2.3%,9.0%


## trip mode share by home location

In [7]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    # num trips by trip type
    trip_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')

    return df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work


df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work = calc_mode_share(trip_county,'hh_county')

In [8]:
df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.7%,44.2%,34.0%,3.3%,16.8%
Kitsap,1.3%,46.0%,37.1%,1.3%,14.4%
Pierce,1.4%,46.0%,39.5%,0.8%,12.4%
Snohomish,1.3%,47.8%,38.4%,0.9%,11.6%


In [9]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"146,497","3,832,507","2,948,147","289,483","1,453,432"
Kitsap,"12,448","456,750","368,105","12,749","143,002"
Pierce,"45,694","1,482,246","1,271,212","24,878","398,128"
Snohomish,"40,489","1,478,426","1,188,577","26,658","359,454"


In [10]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.7%,40.1%,37.2%,3.4%,17.6%
Kitsap,1.3%,42.5%,39.9%,0.9%,15.4%
Pierce,1.5%,41.9%,42.6%,0.7%,13.3%
Snohomish,1.4%,43.4%,41.7%,0.9%,12.6%


In [11]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"125,328","2,979,331","2,766,147","252,909","1,310,349"
Kitsap,"11,352","372,888","350,560","8,092","134,823"
Pierce,"41,683","1,188,575","1,208,980","20,146","377,468"
Snohomish,"36,628","1,167,121","1,122,486","23,971","339,566"


In [12]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.7%,69.0%,14.7%,3.0%,11.6%
Kitsap,1.0%,72.7%,15.2%,4.0%,7.1%
Pierce,1.0%,76.2%,16.2%,1.2%,5.4%
Snohomish,1.0%,77.1%,16.4%,0.7%,4.9%


In [13]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"21,169","853,176","182,000","36,574","143,083"
Kitsap,"1,096","83,862","17,545","4,657","8,179"
Pierce,"4,011","293,671","62,232","4,732","20,660"
Snohomish,"3,861","311,305","66,091","2,687","19,888"


In [14]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rgc, 'hh_rgc')

df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.1%,29.7%,16.3%,6.7%,44.3%
Auburn,2.7%,39.6%,29.2%,3.5%,25.1%
Bellevue,2.6%,32.0%,16.5%,3.6%,45.3%
Bothell Canyon Park,1.4%,52.2%,32.8%,0.6%,13.0%
Bremerton,2.5%,36.0%,22.2%,2.2%,37.0%
Burien,2.0%,44.9%,27.1%,3.0%,23.0%
Everett,2.0%,39.0%,20.5%,2.1%,36.5%
Federal Way,1.9%,36.5%,36.8%,1.9%,22.9%
Greater Downtown Kirkland,1.9%,47.9%,26.0%,3.8%,20.3%


In [15]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"33,689","325,226","177,845","72,947","484,041"
Auburn,224,"3,343","2,471",296,"2,118"
Bellevue,"1,749","21,143","10,889","2,356","29,921"
Bothell Canyon Park,36,"1,356",854,15,339
Bremerton,325,"4,592","2,836",285,"4,713"
Burien,297,"6,697","4,034",449,"3,434"
Everett,520,"10,154","5,343",542,"9,503"
Federal Way,44,851,859,45,533
Greater Downtown Kirkland,604,"15,159","8,243","1,219","6,444"


In [16]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.0%,26.7%,17.9%,7.1%,45.2%
Auburn,2.5%,36.4%,31.2%,3.1%,26.7%
Bellevue,2.6%,28.5%,18.0%,3.9%,47.0%
Bothell Canyon Park,1.3%,48.0%,35.8%,0.5%,14.5%
Bremerton,2.5%,32.5%,23.9%,1.6%,39.6%
Burien,1.9%,40.2%,29.0%,3.2%,25.7%
Everett,2.0%,34.5%,22.1%,2.2%,39.1%
Federal Way,1.9%,32.9%,39.0%,2.0%,24.2%
Greater Downtown Kirkland,1.9%,43.7%,28.3%,3.9%,22.2%


In [17]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"27,537","242,982","163,117","64,440","411,226"
Auburn,191,"2,734","2,347",234,"2,007"
Bellevue,"1,433","15,788","9,986","2,152","26,025"
Bothell Canyon Park,29,"1,059",790,10,320
Bremerton,277,"3,596","2,644",172,"4,390"
Burien,241,"5,125","3,698",407,"3,271"
Everett,441,"7,784","4,988",507,"8,825"
Federal Way,40,681,807,42,500
Greater Downtown Kirkland,509,"11,831","7,667","1,053","6,026"


In [18]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.3%,44.6%,8.0%,4.6%,39.5%
Auburn,3.5%,64.9%,13.2%,6.6%,11.8%
Bellevue,3.0%,50.2%,8.5%,1.9%,36.5%
Bothell Canyon Park,1.8%,75.8%,16.3%,1.3%,4.8%
Bremerton,2.9%,59.6%,11.5%,6.8%,19.3%
Burien,2.6%,72.5%,15.5%,1.9%,7.5%
Everett,2.2%,67.4%,10.1%,1.0%,19.3%
Federal Way,1.5%,64.9%,19.8%,1.1%,12.6%
Greater Downtown Kirkland,2.1%,72.6%,12.6%,3.6%,9.1%


In [19]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"6,152","82,244","14,728","8,507","72,815"
Auburn,33,609,124,62,111
Bellevue,316,"5,355",903,204,"3,896"
Bothell Canyon Park,7,297,64,5,19
Bremerton,48,996,192,113,323
Burien,56,"1,572",336,42,163
Everett,79,"2,370",355,35,678
Federal Way,4,170,52,3,33
Greater Downtown Kirkland,95,"3,328",576,166,418


In [20]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rg, 'hh_rg_proposed')

df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,1.2%,46.1%,40.1%,0.6%,12.0%
Core,1.4%,46.6%,37.8%,1.9%,12.3%
HCT,1.3%,47.1%,38.7%,1.4%,11.5%
Metro,2.1%,40.8%,28.9%,4.5%,23.6%
Rural,0.9%,50.9%,41.8%,0.4%,6.1%
Urban Unincorporated,1.2%,45.9%,42.9%,0.5%,9.5%


In [21]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"16,621","638,813","556,886","8,248","166,522"
Core,"52,939","1,717,337","1,393,306","68,351","455,151"
HCT,"46,631","1,641,042","1,347,117","48,168","401,283"
Metro,"104,043","1,996,118","1,414,035","218,782","1,155,483"
Rural,"17,145","967,050","794,522","7,261","115,779"
Urban Unincorporated,"7,749","289,579","270,187","2,958","59,798"


In [22]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,1.3%,41.7%,43.5%,0.6%,13.0%
Core,1.5%,42.3%,41.1%,1.8%,13.3%
HCT,1.4%,42.8%,42.0%,1.3%,12.5%
Metro,2.1%,36.8%,31.7%,4.6%,24.8%
Rural,1.0%,47.1%,45.2%,0.3%,6.4%
Urban Unincorporated,1.3%,41.6%,46.4%,0.4%,10.3%


In [23]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"15,475","504,572","526,514","6,684","157,919"
Core,"46,986","1,351,367","1,313,189","58,711","426,744"
HCT,"42,137","1,298,656","1,273,312","40,752","378,975"
Metro,"87,028","1,533,096","1,319,768","191,516","1,034,327"
Rural,"16,112","789,160","758,003","5,167","107,284"
Urban Unincorporated,"7,253","231,073","257,399","2,288","56,957"


In [24]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,0.7%,76.3%,17.3%,0.9%,4.9%
Core,1.2%,74.7%,16.3%,2.0%,5.8%
HCT,1.0%,76.0%,16.4%,1.6%,5.0%
Metro,2.4%,64.1%,13.0%,3.8%,16.8%
Rural,0.5%,78.7%,16.2%,0.9%,3.8%
Urban Unincorporated,0.7%,77.7%,17.0%,0.9%,3.8%


In [25]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"1,146","134,241","30,372","1,564","8,603"
Core,"5,953","365,970","80,117","9,640","28,407"
HCT,"4,494","342,386","73,805","7,416","22,308"
Metro,"17,015","463,022","94,267","27,266","121,156"
Rural,"1,033","177,890","36,519","2,094","8,495"
Urban Unincorporated,496,"58,506","12,788",670,"2,841"


In [26]:
equity_geogs = summary_config['hh_equity_geogs']

trip_equity_geog = pd.DataFrame()
df_all = pd.DataFrame()
df_work = pd.DataFrame()
df_non_work = pd.DataFrame()
trip_work = pd.DataFrame()
trip_non_work = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    df['EFA'] = geog
    # trip_equity_geog = pd.concat([trip_equity_geog, df[['equity_geog_type', 'dpurp', 'Mode', 'equity_geog','trexpfac', 'Trip Type']]])

    _df_all, _df_work, _df_non_work, _trip_all, _trip_work, _trip_non_work = calc_mode_share(df, 'equity_geog')

    _df_all['EFA'] = geog
    _df_work['EFA'] = geog
    _df_non_work['EFA'] = geog
    _trip_all['EFA'] = geog
    _trip_work['EFA'] = geog
    _trip_non_work['EFA'] = geog


    df_all = pd.concat([df_all, _df_all])
    df_work = pd.concat([df_work, _df_work])
    df_non_work = pd.concat([df_non_work, _df_non_work])
    trip_all = pd.concat([trip_all, _trip_all])
    trip_work = pd.concat([trip_work, _trip_work])
    trip_non_work = pd.concat([trip_non_work, _trip_non_work])

col_dict = {'hh_efa_dis': 'Disability',
                                'hh_efa_pov200': 'Income',
                                'hh_efa_poc': 'People of Color',
                                'hh_efa_lep': 'LEP',
                                'hh_efa_older': 'Older Adult',
                                'hh_efa_youth': 'Youth'}
col_list = ['EFA','Drove Alone', 'Shared Ride', 'Transit','Walk', 'Bike']

df_all['EFA'] = df_all['EFA'].map(col_dict)
df_all = df_all[col_list]
df_work['EFA'] = df_work['EFA'].map(col_dict)
df_work = df_work[col_list]
df_non_work['EFA'] = df_non_work['EFA'].map(col_dict)
df_non_work = df_non_work[col_list]
trip_all['EFA'] = trip_all['EFA'].map(col_dict)
trip_all = trip_all[col_list]
trip_work['EFA'] = trip_work['EFA'].map(col_dict)
trip_work = trip_work[col_list]
trip_non_work['EFA'] = trip_non_work['EFA'].map(col_dict)
trip_non_work = trip_non_work[col_list]

In [27]:
df_all

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,45.4%,36.1%,2.1%,14.8%,1.5%
Below Regional Average,Disability,45.8%,36.6%,2.3%,13.8%,1.5%
Higher Share of Equity Population,Disability,43.7%,34.5%,2.2%,18.0%,1.6%
Above Regional Average,Older Adult,46.3%,36.8%,2.0%,13.6%,1.4%
Below Regional Average,Older Adult,44.4%,35.9%,2.4%,15.7%,1.7%
Higher Share of Equity Population,Older Adult,47.2%,35.6%,2.1%,13.7%,1.3%
Above Regional Average,LEP,45.6%,36.9%,2.1%,13.9%,1.5%
Below Regional Average,LEP,45.5%,35.5%,2.1%,15.3%,1.5%
Higher Share of Equity Population,LEP,44.6%,37.6%,2.6%,13.7%,1.6%


In [28]:
df_non_work

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,41.4%,39.2%,2.1%,15.7%,1.5%
Below Regional Average,Disability,41.5%,40.0%,2.3%,14.7%,1.5%
Higher Share of Equity Population,Disability,40.0%,37.3%,2.2%,18.9%,1.6%
Above Regional Average,Older Adult,42.3%,39.9%,2.0%,14.4%,1.4%
Below Regional Average,Older Adult,40.0%,39.2%,2.4%,16.7%,1.7%
Higher Share of Equity Population,Older Adult,43.7%,38.4%,2.0%,14.6%,1.3%
Above Regional Average,LEP,41.4%,40.2%,2.1%,14.7%,1.5%
Below Regional Average,LEP,41.5%,38.7%,2.1%,16.2%,1.6%
Higher Share of Equity Population,LEP,40.2%,40.8%,2.7%,14.7%,1.6%


In [29]:
df_work

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,72.2%,15.6%,2.1%,8.7%,1.4%
Below Regional Average,Disability,72.4%,15.3%,2.4%,8.6%,1.4%
Higher Share of Equity Population,Disability,70.1%,14.7%,2.2%,11.5%,1.5%
Above Regional Average,Older Adult,73.1%,15.5%,2.0%,8.2%,1.2%
Below Regional Average,Older Adult,71.1%,15.2%,2.4%,9.7%,1.6%
Higher Share of Equity Population,Older Adult,73.6%,15.0%,2.7%,7.5%,1.1%
Above Regional Average,LEP,72.3%,15.6%,1.8%,9.0%,1.3%
Below Regional Average,LEP,71.7%,14.9%,2.5%,9.5%,1.4%
Higher Share of Equity Population,LEP,72.8%,16.5%,2.2%,7.0%,1.5%


## Mode Share by Trip Destination

In [30]:
tour_rgc_dest = pd.read_csv(output_path / 'agg/dash/tour_rgc_dest.csv')
trip_rgc_dest = pd.read_csv(output_path / 'agg/dash/trip_rgc_dest.csv')

# order RGCs

# all individual centers
l = tour_rgc_dest['tour_d_rgc'].sort_values().unique().tolist()
l.remove('Not in RGC')
l.remove(np.nan)

# aggregated centers
geog_order = ["Region","In RGC","Not in RGC"]

# combine
geog_order.extend(l)

In [31]:
def get_rgc_location(df, dest_col):
    # regional data
    df_region = df.copy()
    df_region[dest_col] = "Region"

    # aggregated centers
    df_isrgc = df.copy()
    df_isrgc.loc[df_isrgc[dest_col]!="Not in RGC", dest_col] = "In RGC"


    df_rgc = pd.concat([
        df_region,
        df_isrgc,
    # 2025 notes:
    # 7 records in the data are missing both mode and and RGC values
    # remove these records
        df.loc[~df[dest_col].isna()]
        ])

    df_rgc[dest_col] = pd.Categorical(df_rgc[dest_col], ordered=True,
                    categories=geog_order)
    
    return df_rgc

df_tour = get_rgc_location(tour_rgc_dest, 'tour_d_rgc')
df_trip = get_rgc_location(trip_rgc_dest, 'trip_d_rgc')




In [32]:
def calc_mode_share_by_rgc(df_trip_geog, geog_col, mode_col, n_trip_col):

    df = df_trip_geog.pivot_table(columns=mode_col, index=geog_col, values=n_trip_col, aggfunc='sum', observed=True)

    df = df.apply(lambda x: x/ df_trip_geog.groupby(geog_col, observed=True)[n_trip_col].sum())

    return df

\[TRIP\] Mode Share for All Purposes by Regional Center (Destination Location)

In [33]:
calc_mode_share_by_rgc(df_trip, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,21.3%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
In RGC,1.9%,15.9%,8.9%,41.9%,0.5%,0.8%,5.6%,24.5%
Not in RGC,1.4%,22.4%,15.1%,45.0%,1.8%,0.2%,1.5%,12.5%
Auburn,1.2%,22.0%,12.4%,48.5%,0.5%,0.2%,3.2%,12.0%
Bellevue,1.7%,15.1%,8.8%,42.8%,0.4%,0.5%,2.6%,28.2%
Bothell Canyon Park,1.0%,22.3%,14.4%,54.0%,1.1%,0.2%,0.4%,6.7%
Bremerton,1.5%,15.8%,9.0%,54.2%,0.1%,0.3%,1.7%,17.3%
Burien,1.1%,22.7%,12.4%,46.6%,0.8%,0.2%,2.2%,13.9%
Everett,1.3%,19.2%,11.1%,47.2%,0.6%,0.3%,1.6%,18.8%


\[TRIP\] Mode Share to Work by Regional Center (Destination Location)

In [34]:
df_trip_work = df_trip[df_trip['dpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_trip_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.4%,9.4%,5.8%,71.6%,0.0%,0.6%,2.3%,8.9%
In RGC,1.8%,8.5%,5.4%,61.9%,0.0%,1.0%,4.9%,16.5%
Not in RGC,1.2%,9.8%,6.1%,76.6%,0.0%,0.3%,0.9%,5.0%
Auburn,1.0%,9.9%,6.7%,74.0%,NaN,0.2%,1.2%,7.1%
Bellevue,1.4%,9.2%,6.0%,65.0%,NaN,0.6%,1.7%,16.0%
Bothell Canyon Park,1.0%,9.5%,5.7%,80.0%,NaN,0.2%,0.2%,3.5%
Bremerton,1.1%,9.9%,6.5%,74.5%,NaN,0.3%,1.6%,6.2%
Burien,1.5%,10.9%,6.0%,73.2%,NaN,0.4%,1.1%,6.9%
Everett,1.2%,9.3%,5.6%,72.4%,NaN,0.3%,0.5%,10.7%


\[TRIP\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [35]:
df_trip_non_work = df_trip[df_trip['dpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_trip_non_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,23.2%,15.4%,40.4%,1.8%,0.3%,2.2%,15.3%
In RGC,1.9%,18.8%,10.3%,34.3%,0.6%,0.7%,5.8%,27.6%
Not in RGC,1.5%,23.9%,16.2%,41.3%,2.0%,0.2%,1.6%,13.4%
Auburn,1.2%,24.2%,13.5%,43.8%,0.6%,0.2%,3.6%,12.9%
Bellevue,1.8%,17.9%,10.2%,32.0%,0.6%,0.4%,3.0%,34.1%
Bothell Canyon Park,1.0%,27.4%,17.9%,43.6%,1.5%,0.2%,0.5%,8.0%
Bremerton,2.0%,21.2%,11.3%,35.5%,0.2%,0.3%,1.9%,27.6%
Burien,1.1%,24.0%,13.1%,43.7%,0.9%,0.2%,2.4%,14.6%
Everett,1.3%,21.9%,12.7%,40.3%,0.8%,0.3%,1.9%,21.0%


## Mode Share by Tour Destination

\[TOUR\] Mode Share for All Purposes by Regional Center (Destination Location)

In [36]:
calc_mode_share_by_rgc(df_tour, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,,
Region,1.4%,22.1%,19.3%,0.1%,37.9%,2.6%,0.5%,3.2%,13.0%
In RGC,1.5%,18.4%,14.2%,0.5%,39.3%,0.6%,1.0%,8.0%,16.3%
Not in RGC,1.3%,23.2%,20.7%,0.1%,37.5%,3.1%,0.3%,1.9%,12.0%
Auburn,0.7%,25.2%,20.6%,0.1%,41.9%,0.3%,0.3%,2.4%,8.6%
Bellevue,1.3%,17.7%,14.4%,0.2%,42.8%,0.3%,0.7%,3.4%,19.1%
Bothell Canyon Park,0.8%,24.2%,21.7%,0.0%,45.0%,1.9%,0.4%,0.8%,5.0%
Bremerton,1.2%,20.9%,15.7%,0.5%,49.9%,0.1%,0.4%,0.8%,10.4%
Burien,0.8%,23.6%,18.3%,0.0%,40.1%,1.6%,0.3%,3.9%,11.3%
Everett,0.9%,22.8%,17.9%,0.1%,40.7%,1.0%,0.4%,2.4%,13.9%


\[TOUR\] Mode Share to Work by Regional Center (Destination Location)

In [37]:
df_tour_work = df_tour[df_tour['pdpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_tour_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.4%,15.3%,12.5%,0.5%,62.2%,1.0%,2.7%,4.3%
In RGC,1.8%,13.9%,11.3%,1.1%,55.1%,1.8%,6.0%,8.8%
Not in RGC,1.2%,16.1%,13.1%,0.2%,66.0%,0.6%,1.0%,1.8%
Auburn,1.0%,16.7%,14.7%,0.2%,62.8%,0.5%,1.6%,2.4%
Bellevue,1.5%,15.3%,13.2%,0.4%,59.3%,1.0%,1.9%,7.4%
Bothell Canyon Park,1.0%,16.0%,12.6%,0.1%,68.7%,0.7%,0.3%,0.7%
Bremerton,0.9%,16.9%,13.6%,0.8%,64.4%,0.5%,1.2%,1.7%
Burien,1.2%,16.6%,13.2%,0.2%,63.8%,0.7%,1.5%,2.8%
Everett,1.2%,16.0%,12.9%,0.2%,63.6%,0.6%,0.5%,4.9%


\[TOUR\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [38]:
df_tour_non_work = df_tour[df_tour['pdpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_tour_non_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.3%,24.7%,21.9%,28.6%,3.5%,0.2%,3.4%,16.3%
In RGC,1.3%,21.9%,16.5%,27.2%,1.1%,0.4%,9.5%,22.1%
Not in RGC,1.3%,25.3%,23.0%,28.9%,4.1%,0.2%,2.2%,15.1%
Auburn,0.6%,28.1%,22.6%,34.7%,0.4%,0.2%,2.6%,10.8%
Bellevue,1.0%,20.9%,16.1%,22.0%,0.7%,0.2%,5.2%,33.9%
Bothell Canyon Park,0.8%,29.5%,27.5%,30.0%,3.1%,0.2%,1.2%,7.8%
Bremerton,1.8%,27.2%,19.1%,27.3%,0.3%,0.3%,0.2%,23.9%
Burien,0.7%,25.0%,19.3%,35.5%,2.0%,0.2%,4.3%,13.0%
Everett,0.8%,25.7%,20.0%,30.9%,1.4%,0.3%,3.2%,17.7%
